In [1]:
from src.recognize import load_database, recognize_song

In [2]:
db = load_database("../songs")
match = recognize_song("../recordings/Bad_org_01.m4a", db)

if match:
    print(f"Matched: {match[0]} (score: {match[1]})")
else:
    print("No match found.")

Matched: I Did Something Bad.m4a (score: 62)
